In [1]:
import requests
import config
import pandas as pd
import numpy as np
from datetime import date
import pickle
import time
import sqlite3

In [2]:
URL = {
    'base': 'https://{proxy}.api.riotgames.com/lol/{url}',
    'summonerData': 'summoner/v{version}/summoners/by-name/{sumName}?api_key={apiKey}',
    'leagueData': 'league/v{version}/entries/{queueType}/{tier}/{rank}?api_key={apiKey}'
}

VERSION = {
    'summonerData': '4',
    'leagueData': '4'
}

In [3]:
queueTypes = ['RANKED_SOLO_5x5', 'RANKED_FLEX_SR', 'RANKED_FLEX_TT']
tiers = {'DIAMOND': 1, 'PLATINUM': 2, 'GOLD': 3, 'SILVER': 4, 'BRONZE': 5, 'IRON': 6}
ranks = {'I': 1, 'II': 2, 'III': 3, 'IV': 4}

In [4]:
class riotApi(object):
    
    def __init__(self, apiKey, region = 'na1'):
        self.apiKey = apiKey
        self.region = region
        
    def _request(self, apiUrl, params = {}):
        args = {'apiKey': self.apiKey}
        for key, value in params.items():
            if key not in args:
                args[key] = value
        response = requests.get(
            URL['base'].format(
                proxy = self.region,
                url = apiUrl),
            params = args)
        return response.json()
    
    def summonerData(self, name):
        apiUrl = URL['summonerData'].format(
            version = VERSION['summonerData'],
            sumName = name,
            apiKey = self.apiKey)
        return self._request(apiUrl)
    
    def leagueData(self, queueType, tier, rank):
        apiUrl = URL['leagueData'].format(
            version = VERSION['leagueData'],
            queueType = queueType,
            tier = tier,
            rank = rank,
            apiKey = self.apiKey)
        return self._request(apiUrl)

In [43]:
class sqlDatabase():
    
    def __init__(self, path):
        self.path = path
    
    def _open(self):
        self.db = sqlite3.connect(self.path)
        self.cursor = self.db.cursor()
        
    def _close(self):
        self.cursor.close()
        self.db.close()
        
    def loadData(self, df, table):
        self._open()
        # Get columns types from dataframe
        types = {np.dtype('O'): 'text',
            np.dtype('int64'): 'number',
            np.dtype('bool'): 'number'}
        
        colTypes = [types[x] for x in list(df.dtypes)]
        colNames = list(df.columns)
        columns = ''

        for column in zip(colNames, colTypes):
            columns += column[0] + ' ' + column[1] + ', '
        
        # Create summoners table in the database
        try:
            self.cursor.execute(f'CREATE TABLE SUMMONERS ({columns[:-2]})')
            self.db.commit()
        except:
            pass
        
        # Add scraped data to the database
        df.to_sql(table, self.db, if_exists = 'append', index = False)
        self._close()

In [6]:
api = riotApi(config.apiKey)
leagueRequest = api.leagueData(queueTypes[0], 'DIAMOND', 'I')
summonerDF = pd.DataFrame(leagueRequest)
summonerDF['date'] = date.today()

levelSR = []
for name in summonerDF['summonerName']:
    try:
        summonerLevel = api.summonerData(name)['summonerLevel']
        levelSR.append(summonerLevel)
    except KeyError:
        time.sleep(60*2)
        summonerLevel = api.summonerData(name)['summonerLevel']
        levelSR.append(summonerLevel)
summonerDF['level'] = levelSR

for key, val in ranks.items():
    summonerDF.loc[summonerDF['rank'] == key, 'rank'] = val
    
for key, val in tiers.items():
    summonerDF.loc[summonerDF['tier'] == key, 'tier'] = val

summonerDF = summonerDF.drop(columns = ['miniSeries'])

In [38]:
db_lol = sqlite3.connect('../data/test.db')
sqlCursor = db_lol.cursor()

# Get columns types from dataframe
types = {np.dtype('O'): 'text',
        np.dtype('int64'): 'number',
        np.dtype('bool'): 'number'}
colTypes = [types[x] for x in list(summonerDF.dtypes)]
colNames = list(summonerDF.columns)
columns = ''

for column in zip(colNames, colTypes):
    columns += column[0] + ' ' + column[1] + ', '

    
# Create summoners table
try:
    sqlCursor.execute(f'CREATE TABLE SUMMONERS ({columns[:-2]})')
    db_lol.commit()
except:
    pass

# Add scraped data to the database
summonerDF.to_sql('SUMMONERS', db_lol,
                  if_exists = 'append', index = False)

sqlCursor.close()
db_lol.close()